In [ ]:
import numpy as np
from submit import my_map
from submit import my_fit
from submit import my_decode
import time as tm

In [ ]:
Z_trn = np.loadtxt( "secret_trn.txt" )
Z_tst = np.loadtxt( "secret_tst.txt" )

n_trials = 5

d_size = 0
t_train = 0
t_map = 0
acc = 0

In [ ]:
for t in range( n_trials ):
  tic = tm.perf_counter()
  w, b = my_fit( Z_trn[:, :-1], Z_trn[:,-1] )
  toc = tm.perf_counter()

  t_train += toc - tic
  w = w.reshape( -1 )

  d_size += w.shape[0]

  tic = tm.perf_counter()
  feat = my_map( Z_tst[:, :-1] )
  toc = tm.perf_counter()
  t_map += toc - tic

  scores = feat.dot( w ) + b

  pred = np.zeros_like( scores )
  pred[ scores > 0 ] = 1

  acc += np.average( Z_tst[ :, -1 ] == pred )

In [ ]:
d_size /= n_trials
t_train /= n_trials
t_map /= n_trials
acc /= n_trials

In [ ]:
def get_model( p, q, r, s ):
  p = np.maximum( p, 0 )
  q = np.maximum( q, 0 )
  r = np.maximum( r, 0 )
  s = np.maximum( s, 0 )
  d = p - q
  c = r - s
  alpha = ( d + c ) / 2
  beta = ( d - c ) / 2
  w = np.zeros( ( len( alpha ) + 1, )  )
  w[:-1] += alpha
  w[1:] += beta
  return w

In [ ]:
W = np.loadtxt( "secret_mod.txt" )
( n_models, dims ) = W.shape
t_decode = 0
m_dist = 0
for t in range( n_trials ):
  for itr in range( n_models ):
    w = W[ itr, : ]
    tic = tm.perf_counter()
    p_hat, q_hat, r_hat, s_hat = my_decode( w )
    toc = tm.perf_counter()
    t_decode += toc - tic
    w_hat = get_model( p_hat, q_hat, r_hat, s_hat )
    m_dist += np.linalg.norm( w - w_hat )

In [ ]:
t_decode /= ( n_trials * n_models )
m_dist /= ( n_trials * n_models )

In [ ]:
print( f"{d_size},{t_train},{t_map},{1 - acc},{t_decode},{m_dist}" )

8.0,9.857800023382879e-06,0.0002463024000007863,0.31499999999999995,1.3458400007948513e-05,1.6199987057689555
